In [1]:
import polars as pl

In [2]:
#Call datawars2 api and create a dataframe
url = "https://api.datawars2.ie/gw2/v1/items/csv?filter=&fields=id,name,chat_link,type,rarity,weight,weaponType,level,name_de,buy_price,sell_price,vendor_value,upgrade1,upgradeName,NoSalvage,profit,lastUpdate,charm,AccountBound"
df_api = pl.read_csv(url)
#Read unid gear csv file and create a dataframe 
df_pl=pl.read_csv("paintball_green_unid.csv")

In [3]:
#Group items by id and sum their count.
filtered_unid_Green = (
    df_pl
    .groupby("item_id")
    .agg(pl.col("count").sum())
    .sort("count")
    
)
print(filtered_unid_Green)
#Check if all items are 
print(filtered_unid_Green["count"].sum())

shape: (834, 2)
┌─────────┬────────┐
│ item_id ┆ count  │
│ ---     ┆ ---    │
│ i64     ┆ i64    │
╞═════════╪════════╡
│ 84731   ┆ -49294 │
│ 26496   ┆ 1      │
│ 2552    ┆ 1      │
│ 26872   ┆ 1      │
│ ...     ┆ ...    │
│ 25921   ┆ 167    │
│ 1243    ┆ 169    │
│ 1249    ┆ 169    │
│ 26465   ┆ 177    │
└─────────┴────────┘
0


In [4]:
#Merge unid gear dataframe with api dataframe
merged_with_api = filtered_unid_Green.join(df_api, left_on="item_id", right_on="id")
merged_with_api.limit(5)

item_id,count,name,chat_link,type,rarity,weight,weaponType,level,name_de,buy_price,sell_price,vendor_value,upgrade1,upgradeName,NoSalvage,profit,lastUpdate,charm,AccountBound
i64,i64,str,str,str,str,str,str,i64,str,i64,i64,i64,i64,str,bool,i64,str,str,bool
80,1,"""Nika's Mask""","""[&AgFQAAAA]""","""Armor""","""Exotic""","""Medium""","""Helm""",80,"""Nika-Maske""",3236,4985,330,24703,"""of Infiltratio...",null,1001,"""2023-04-13T16:...",null,false
82,1,"""Berserker's Se...","""[&AgFSAAAA]""","""Armor""","""Exotic""","""Light""","""Coat""",78,"""Berserkerhafte...",1419,1923,384,24687,"""of the Afflict...",null,216,"""2023-04-13T16:...",null,false
92,1,"""Carrion Sneakt...","""[&AgFcAAAA]""","""Armor""","""Exotic""","""Medium""","""Helm""",79,"""Faulende Leise...",1017,1717,325,24797,"""of the Flame L...",null,442,"""2023-04-13T16:...",null,false
131,1,"""Khilbron's Coa...","""[&AgGDAAAA]""","""Armor""","""Exotic""","""Light""","""Coat""",80,"""Khilbron-Wams""",2599,2893,396,24688,"""of the Lich""",null,-140,"""2023-04-13T16:...",null,false
136,1,"""Vatlaaw's Mask...","""[&AgGIAAAA]""","""Armor""","""Exotic""","""Medium""","""Helm""",80,"""Vatlaaw-Maske""",2338,2495,330,24797,"""of the Flame L...",null,-217,"""2023-04-13T16:...",null,false


In [5]:
#Split the merged data frame into several data frames depending on the rarity of the item
exotic_item_drops = merged_with_api.filter(pl.col("rarity") == "Exotic")
rare_item_drops = merged_with_api.filter(pl.col("rarity") == "Rare")
masterwork_item_drops = merged_with_api.filter(pl.col("rarity") == "Masterwork").filter(
    pl.col("type") != "Container"
)
#Total number of opened unid gears
opened_unid_gear = abs(
    filtered_unid_Green.filter(pl.col("item_id") == 84731).select("count").item()
)

# Drops overview

In [6]:
print("Opend green unid gear: " + str(opened_unid_gear))
print("Outcome: ")
#Print overall count of masterwork items
print(
    str(masterwork_item_drops["count"].sum())
    + " Masterwork items. That are "
    + str(round(masterwork_item_drops["count"].sum() / opened_unid_gear * 100, 2))
    + "%"
)
#Print overall count of rare items
print(
    str(rare_item_drops["count"].sum())
    + " Rare items. That are "
    + str(round(rare_item_drops["count"].sum() / opened_unid_gear * 100, 2))
    + "%"
)
#Print overall count of exotic items
print(
    str(exotic_item_drops["count"].sum())
    + " Exotic items. That are "
    + str(round(exotic_item_drops["count"].sum() / opened_unid_gear * 100, 2))
    + "%"
)

Opend green unid gear: 49294
Outcome: 
47548 Masterwork items. That are 96.46%
1623 Rare items. That are 3.29%
123 Exotic items. That are 0.25%


In [7]:
#Display description of the different data frames
print("Masterwork items distribution")
print(masterwork_item_drops["count"].describe())
print("Rare items distribution")
print(rare_item_drops["count"].describe())
print("Exotic items distribution")
print(exotic_item_drops["count"].describe())

Masterwork items distribution
shape: (6, 2)
┌────────────┬────────────┐
│ statistic  ┆ value      │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ min        ┆ 105.0      │
│ max        ┆ 177.0      │
│ null_count ┆ 0.0        │
│ mean       ┆ 142.359281 │
│ std        ┆ 12.160203  │
│ count      ┆ 334.0      │
└────────────┴────────────┘
Rare items distribution
shape: (6, 2)
┌────────────┬──────────┐
│ statistic  ┆ value    │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ min        ┆ 1.0      │
│ max        ┆ 15.0     │
│ null_count ┆ 0.0      │
│ mean       ┆ 4.140306 │
│ std        ┆ 2.093275 │
│ count      ┆ 392.0    │
└────────────┴──────────┘
Exotic items distribution
shape: (6, 2)
┌────────────┬──────────┐
│ statistic  ┆ value    │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ min        ┆ 1.0      │
│ max        ┆ 3.0      │
│ null_count ┆ 0.0      │
│ mean       ┆ 1.14953

## Check for precursor drops

In [8]:
"""
Tooth of Frostfang, Spark, Bard, Dawn,  Colossus, Carcharias, Leaf of Kudzu, The Energizer, Chaos Gun, Hunter, Storm, The Chosen,
Lover, Rage, Legend,Zap, Rodgort's Flame, Venom, Howl,Dusk
"""
precursor_id_list_gen1 = [
    219166,
    29167,
    29168,
    29169,
    29170,
    29171,
    29172,
    29173,
    29174,
    29175,
    29176,
    29177,
    29178,
    29179,
    29180,
    29181,
    29182,
    29183,
    29184,
    29185,
]

merged_with_api.filter(pl.col("item_id").is_in(precursor_id_list_gen1))

item_id,count,name,chat_link,type,rarity,weight,weaponType,level,name_de,buy_price,sell_price,vendor_value,upgrade1,upgradeName,NoSalvage,profit,lastUpdate,charm,AccountBound
i64,i64,str,str,str,str,str,str,i64,str,i64,i64,i64,i64,str,bool,i64,str,str,bool


# Droprates of Items grouped by rarity, type, weapon type and weight

In [9]:
#Function to calculate the overall sell/buy price value after tax or the overall value of the vendor value
def compute_value(column: str, tax: bool) -> pl.Expr:
    if tax:
        return (pl.col(column) * 0.85 * pl.col("count")).sum()
    else:
        return (pl.col(column) * pl.col("count")).sum()

#Function to calculate the avg value of the grouped values and set their alias
def compute_avg_value(column: str) -> pl.Expr:
    alias_string = "avg_" + column + "_in_copper"
    return (pl.col(column) / pl.col("count")).alias(alias_string)

#Calculate avg sell, avg buy and avg vendor value and create now columns with this rows.
def edit_dataframe(df: pl.DataFrame):
    return (
        df.groupby(["rarity", "type", "weaponType", "weight"])
        .agg(
            [
                (pl.col("count").sum()),
                compute_value("sell_price", True),
                compute_value("buy_price", True),
                compute_value("vendor_value", False),
            ]
        )
        .sort(["type", "weaponType", "weight", "count"])
        .with_columns(
            [
                (pl.col("count") / opened_unid_gear * 100).alias("ratio in %"),
                compute_avg_value("sell_price"),
                compute_avg_value("buy_price"),
                compute_avg_value("vendor_value"),
            ]
        )
        .drop(["sell_price", "buy_price", "vendor_value"])
    )

In [10]:
#Change polars settings to show 100 rows and 10 columns
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(10)
#Show grouped masterwork dataframe
masterwork_items_grouped = edit_dataframe(masterwork_item_drops)
print(masterwork_items_grouped)

shape: (39, 9)
┌──────────┬─────────┬──────────┬────────┬───────┬──────────┬────────────┬────────────┬────────────┐
│ rarity   ┆ type    ┆ weaponTy ┆ weight ┆ count ┆ ratio in ┆ avg_sell_p ┆ avg_buy_pr ┆ avg_vendor │
│ ---      ┆ ---     ┆ pe       ┆ ---    ┆ ---   ┆ %        ┆ rice_in_co ┆ ice_in_cop ┆ _value_in_ │
│ str      ┆ str     ┆ ---      ┆ str    ┆ i64   ┆ ---      ┆ pper       ┆ per        ┆ copper     │
│          ┆         ┆ str      ┆        ┆       ┆ f64      ┆ ---        ┆ ---        ┆ ---        │
│          ┆         ┆          ┆        ┆       ┆          ┆ f64        ┆ f64        ┆ f64        │
╞══════════╪═════════╪══════════╪════════╪═══════╪══════════╪════════════╪════════════╪════════════╡
│ Masterwo ┆ Armor   ┆ Boots    ┆ Heavy  ┆ 1276  ┆ 2.58855  ┆ 130.752782 ┆ 128.555172 ┆ 130.12069  │
│ rk       ┆         ┆          ┆        ┆       ┆          ┆            ┆            ┆            │
│ Masterwo ┆ Armor   ┆ Boots    ┆ Light  ┆ 1245  ┆ 2.525662 ┆ 152.780161 ┆ 1

In [11]:
#Show grouped rare dataframe
rare_items_grouped = edit_dataframe(rare_item_drops)
print(rare_items_grouped)

shape: (39, 9)
┌────────┬─────────┬──────────┬────────┬───────┬────────────┬────────────┬────────────┬────────────┐
│ rarity ┆ type    ┆ weaponTy ┆ weight ┆ count ┆ ratio in % ┆ avg_sell_p ┆ avg_buy_pr ┆ avg_vendor │
│ ---    ┆ ---     ┆ pe       ┆ ---    ┆ ---   ┆ ---        ┆ rice_in_co ┆ ice_in_cop ┆ _value_in_ │
│ str    ┆ str     ┆ ---      ┆ str    ┆ i64   ┆ f64        ┆ pper       ┆ per        ┆ copper     │
│        ┆         ┆ str      ┆        ┆       ┆            ┆ ---        ┆ ---        ┆ ---        │
│        ┆         ┆          ┆        ┆       ┆            ┆ f64        ┆ f64        ┆ f64        │
╞════════╪═════════╪══════════╪════════╪═══════╪════════════╪════════════╪════════════╪════════════╡
│ Rare   ┆ Armor   ┆ Boots    ┆ Heavy  ┆ 50    ┆ 0.101432   ┆ 924.154    ┆ 833.442    ┆ 194.64     │
│ Rare   ┆ Armor   ┆ Boots    ┆ Light  ┆ 39    ┆ 0.079117   ┆ 896.553846 ┆ 845.270513 ┆ 194.923077 │
│ Rare   ┆ Armor   ┆ Boots    ┆ Medium ┆ 48    ┆ 0.097375   ┆ 909.446875 ┆ 8

In [12]:
#Show grouped exotic dataframe
exotic_items_grouped = edit_dataframe(exotic_item_drops)
print(exotic_items_grouped)

shape: (38, 9)
┌────────┬─────────┬──────────┬────────┬───────┬────────────┬────────────┬────────────┬────────────┐
│ rarity ┆ type    ┆ weaponTy ┆ weight ┆ count ┆ ratio in % ┆ avg_sell_p ┆ avg_buy_pr ┆ avg_vendor │
│ ---    ┆ ---     ┆ pe       ┆ ---    ┆ ---   ┆ ---        ┆ rice_in_co ┆ ice_in_cop ┆ _value_in_ │
│ str    ┆ str     ┆ ---      ┆ str    ┆ i64   ┆ f64        ┆ pper       ┆ per        ┆ copper     │
│        ┆         ┆ str      ┆        ┆       ┆            ┆ ---        ┆ ---        ┆ ---        │
│        ┆         ┆          ┆        ┆       ┆            ┆ f64        ┆ f64        ┆ f64        │
╞════════╪═════════╪══════════╪════════╪═══════╪════════════╪════════════╪════════════╪════════════╡
│ Exotic ┆ Armor   ┆ Boots    ┆ Heavy  ┆ 1     ┆ 0.002029   ┆ 1530.0     ┆ 912.9      ┆ 264.0      │
│ Exotic ┆ Armor   ┆ Boots    ┆ Medium ┆ 1     ┆ 0.002029   ┆ 2380.0     ┆ 1907.4     ┆ 264.0      │
│ Exotic ┆ Armor   ┆ Coat     ┆ Heavy  ┆ 6     ┆ 0.012172   ┆ 2069.60833 ┆ 1

In [13]:
#Show avg copper values of masterwork, rare and exotic
print(
    "Avg vendor value of masterwork items dropped by green unid gear: "
    + str(
        round(
            (
                masterwork_items_grouped["avg_vendor_value_in_copper"]
                * (masterwork_items_grouped["ratio in %"] / 100)
            ).sum(),
            2,
        )
    )
    + " copper"
)
print(
    "Based on the selling price, the impact of rare items on the outcome of green unid gear: "
    + str(
        round(
            (
                rare_items_grouped["avg_sell_price_in_copper"]
                * (rare_items_grouped["ratio in %"] / 100)
            ).sum(),
            2,
        )
    )
    + " copper"
)
print(
    "Based on the selling price, the impact of exotic items on the outcome of green unid gear: "
    + str(
        round(
            (
                exotic_items_grouped["avg_sell_price_in_copper"]
                * (exotic_items_grouped["ratio in %"] / 100)
            ).sum(),
            2,
        )
    )
    + " copper"
)

Avg vendor value of masterwork items dropped by green unid gear: 153.08 copper
Based on the selling price, the impact of rare items on the outcome of green unid gear: 31.82 copper
Based on the selling price, the impact of exotic items on the outcome of green unid gear: 8.02 copper


# Lets get runes and sigills

In [14]:
#Function to create a upgrade dataframe from item drop dataframe
def create_upgrade_dataframe(dataframe: pl.DataFrame):
    return(
         dataframe.groupby("upgrade1")
    .agg(pl.col("count").sum())
    .join(df_api, left_on="upgrade1", right_on="id")[
        ["upgrade1","name","count", "buy_price", "sell_price", "vendor_value", "charm"]
    ]
    .with_columns(
        [
            (pl.col("count") / opened_unid_gear * 100).alias("ratio in %"),
            pl.col("sell_price") * 0.85,
            pl.col("buy_price") * 0.85,
        ]
    )
    )


In [15]:
# Group masterwork items by upgrade id and sum them.
masterwork_upgrade = create_upgrade_dataframe(masterwork_item_drops)

#Show data and statistics
print(masterwork_upgrade)
print(masterwork_upgrade["count"].describe())
print(masterwork_upgrade["ratio in %"].sum())
print(
    "upgrades: "
    + str(round((masterwork_upgrade["sell_price"] * (masterwork_upgrade["ratio in %"] / 100)).sum(), 2))
    + " copper"
)
print(
    "upgrades buy price: "
    + str(round((masterwork_upgrade["buy_price"] * (masterwork_upgrade["ratio in %"] / 100)).sum(), 2))
    + " copper"
)

shape: (47, 8)
┌──────────┬──────────────┬───────┬───────────┬────────────┬────────────┬─────────────┬────────────┐
│ upgrade1 ┆ name         ┆ count ┆ buy_price ┆ sell_price ┆ vendor_val ┆ charm       ┆ ratio in % │
│ ---      ┆ ---          ┆ ---   ┆ ---       ┆ ---        ┆ ue         ┆ ---         ┆ ---        │
│ i64      ┆ str          ┆ i64   ┆ f64       ┆ f64        ┆ ---        ┆ str         ┆ f64        │
│          ┆              ┆       ┆           ┆            ┆ i64        ┆             ┆            │
╞══════════╪══════════════╪═══════╪═══════════╪════════════╪════════════╪═════════════╪════════════╡
│ 24547    ┆ Minor Sigil  ┆ 740   ┆ 62.9      ┆ 68.0       ┆ 16         ┆ Pain        ┆ 1.501197   │
│          ┆ of Fire      ┆       ┆           ┆            ┆            ┆             ┆            │
│ 24549    ┆ Minor Sigil  ┆ 692   ┆ 113.9     ┆ 160.65     ┆ 16         ┆ Control     ┆ 1.403822   │
│          ┆ of Water     ┆       ┆           ┆            ┆            ┆   

In [16]:
# Group rare items by upgrade id and sum them.
rare_upgrade=create_upgrade_dataframe(rare_item_drops)
#Show data and statistics
print(rare_upgrade)
print(rare_upgrade["count"].describe())
print(rare_upgrade["ratio in %"].sum())
print(
    "upgrades: "
    + str(round((rare_upgrade["sell_price"] * (rare_upgrade["ratio in %"] / 100)).sum(), 2))
    + " copper"
)
print(
    "upgrades buy price: "
    + str(round((rare_upgrade["buy_price"] * (rare_upgrade["ratio in %"] / 100)).sum(), 2))
    + " copper"
)

shape: (53, 8)
┌──────────┬──────────────┬───────┬───────────┬────────────┬────────────┬─────────────┬────────────┐
│ upgrade1 ┆ name         ┆ count ┆ buy_price ┆ sell_price ┆ vendor_val ┆ charm       ┆ ratio in % │
│ ---      ┆ ---          ┆ ---   ┆ ---       ┆ ---        ┆ ue         ┆ ---         ┆ ---        │
│ i64      ┆ str          ┆ i64   ┆ f64       ┆ f64        ┆ ---        ┆ str         ┆ f64        │
│          ┆              ┆       ┆           ┆            ┆ i64        ┆             ┆            │
╞══════════╪══════════════╪═══════╪═══════════╪════════════╪════════════╪═════════════╪════════════╡
│ 24546    ┆ Major Sigil  ┆ 23    ┆ 108.8     ┆ 170.0      ┆ 108        ┆ Pain        ┆ 0.046659   │
│          ┆ of Fire      ┆       ┆           ┆            ┆            ┆             ┆            │
│ 24550    ┆ Major Sigil  ┆ 74    ┆ 198.05    ┆ 198.9      ┆ 108        ┆ Control     ┆ 0.15012    │
│          ┆ of Water     ┆       ┆           ┆            ┆            ┆   